In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [2]:
origin_test_path ="../../data/test.csv"
origin_train_path = "../../data/train.csv"

In [3]:
dt = pd.read_csv(origin_train_path)
dt_test = pd.read_csv(origin_test_path)

In [4]:
apart_code_path="../../data/apart_cod.csv"

In [5]:
apart_code_df = pd.read_csv(apart_code_path)

In [6]:
concat = pd.concat([dt, dt_test])

In [7]:
apart_code_df['단지명'] = apart_code_df['단지명_건축물대장'].str.replace('아파트', '', regex=False)
apart_code_df['단지고유번호'] = apart_code_df['단지고유번호'].astype(str)

In [8]:
apart_code_df = apart_code_df.rename(columns={"동수": "k-전체동수"})

In [10]:
concat = concat.merge(apart_code_df[['단지명', '단지고유번호', 'k-전체동수', '세대수', "사용승인일"]],
                      left_on='아파트명',
                      right_on='단지명',
                      how='left')

In [11]:
concat.head(3)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,관리비 업로드,좌표X,좌표Y,단지신청일,target,단지명,단지고유번호,k-전체동수_y,세대수,사용승인일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000.0,개포6차우성,11680100001448,8.0,270.0,1987-11-21
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,N,127.05721,37.476763,2022-11-17 10:19:06.0,123500.0,개포6차우성,11680100001448,8.0,270.0,1987-11-21
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,N,127.05721,37.476763,2022-11-17 10:19:06.0,91500.0,개포6차우성,11680100001448,8.0,270.0,1987-11-21


In [12]:
concat.isna().sum()

시군구                              0
번지                             227
본번                              75
부번                              75
아파트명                      15120744
전용면적(㎡)                          0
계약년월                             0
계약일                              0
층                                0
건축년도                             0
도로명                              0
해제사유발생일                   26369174
등기신청일자                           0
거래유형                             0
중개사소재지                           0
k-단지분류(아파트,주상복합등등)        26126899
k-전화번호                    26126061
k-팩스번호                    26128642
단지소개기존clob                26512608
k-세대타입(분양형태)              26125336
k-관리방식                    26125336
k-복도유형                    26125665
k-난방방식                    26125336
k-전체동수_x                  26126501
k-전체세대수                   26125336
k-건설사(시공사)                26127210
k-시행사                     26127407
k-사용검사일-사용승인일             26125470
k-연면적               